In [11]:
from pathlib import Path
import json

import cv2
import numpy as np
import pandas as pd
import gradio as gr
from ultralytics import YOLO


In [12]:
ROOT = Path().resolve().parent
DATA = ROOT / "data"

IM_DIR  = DATA / "images"
ROI_DIR = DATA / "roi"
RES_DIR = DATA / "results"

IM_DIR.mkdir(parents=True, exist_ok=True)
ROI_DIR.mkdir(parents=True, exist_ok=True)
RES_DIR.mkdir(parents=True, exist_ok=True)

print("ROOT   :", ROOT)
print("IM_DIR :", IM_DIR)
print("ROI_DIR:", ROI_DIR)
print("RES_DIR:", RES_DIR)


ROOT   : E:\Adri\Universidad\II 2025\TOPICOS\Proyecto\parking-vision
IM_DIR : E:\Adri\Universidad\II 2025\TOPICOS\Proyecto\parking-vision\data\images
ROI_DIR: E:\Adri\Universidad\II 2025\TOPICOS\Proyecto\parking-vision\data\roi
RES_DIR: E:\Adri\Universidad\II 2025\TOPICOS\Proyecto\parking-vision\data\results


Modelo YOLO y clases de interés

In [13]:
MODEL_WEIGHTS = "yolov8n.pt"
print("[INFO] Cargando modelo YOLO:", MODEL_WEIGHTS)
model = YOLO(MODEL_WEIGHTS)

VEHICLE_CLASSES = {2, 3, 5, 7}


[INFO] Cargando modelo YOLO: yolov8n.pt


In [14]:
def load_rois(image_name: str):
    stem = Path(image_name).stem
    roi_path = ROI_DIR / f"{stem}_rois.json"
    if not roi_path.exists():
        raise FileNotFoundError(f"No se encontró el archivo de ROIs: {roi_path}")

    with open(roi_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    roi_defs = []

    for r in data:
        pid = str(
            r.get("plaza_id")
            or r.get("id")
            or r.get("slot_id")
            or "?"
        )

        if "bbox" in r:
            x1, y1, x2, y2 = r["bbox"]

        elif {"x1", "y1", "x2", "y2"}.issubset(r.keys()):
            x1, y1, x2, y2 = r["x1"], r["y1"], r["x2"], r["y2"]

        elif {"x", "y", "w", "h"}.issubset(r.keys()):
            x1 = r["x"]
            y1 = r["y"]
            x2 = x1 + r["w"]
            y2 = y1 + r["h"]

        elif "points" in r:
            pts = r["points"]
            if not pts or len(pts) < 2:
                raise ValueError(f"ROI con 'points' inválido en {roi_path}: {pts}")
            xs = [p[0] for p in pts]
            ys = [p[1] for p in pts]
            x1, y1 = min(xs), min(ys)
            x2, y2 = max(xs), max(ys)

        else:
            raise ValueError(
                f"Formato de ROI desconocido en {roi_path}. "
                f"Claves disponibles: {list(r.keys())}"
            )

        roi_defs.append(
            {
                "plaza_id": pid,
                "bbox": [int(x1), int(y1), int(x2), int(y2)],
            }
        )

    print(f"[INFO] Cargados {len(roi_defs)} ROIs desde {roi_path.name}")
    return roi_defs


Detección de vehículos

In [15]:
def detect_vehicles(frame, conf: float = 0.45):
    results = model.predict(
        frame,
        conf=conf,
        imgsz=960,
        verbose=False,
    )

    boxes_out = []
    if not results:
        return boxes_out

    res = results[0]
    if res.boxes is None:
        return boxes_out

    for box, cls in zip(res.boxes.xyxy, res.boxes.cls):
        cls_id = int(cls)
        if cls_id not in VEHICLE_CLASSES:
            continue

        x1, y1, x2, y2 = box.tolist()
        boxes_out.append([int(x1), int(y1), int(x2), int(y2)])

    print(f"[DEBUG] Vehículos detectados: {len(boxes_out)}")
    return boxes_out


In [16]:
def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interW = max(0, xB - xA)
    interH = max(0, yB - yA)
    interArea = interW * interH
    if interArea == 0:
        return 0.0

    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    return interArea / float(boxAArea + boxBArea - interArea)


def occupancy_status(boxes, roi_defs, frame_shape, thr: float = 0.10):
    status = {}
    for roi in roi_defs:
        pid = roi["plaza_id"]
        rb = roi["bbox"]
        max_iou = 0.0
        for vb in boxes:
            max_iou = max(max_iou, iou(rb, vb))
        status[pid] = "Ocupada" if max_iou >= thr else "Libre"
    return status


In [17]:
def draw_overlay(frame, roi_defs, status, boxes):
    vis = frame.copy()

    for (x1, y1, x2, y2) in boxes:
        cv2.rectangle(vis, (x1, y1), (x2, y2), (0, 255, 255), 2)

    libres = 0
    total = len(roi_defs)

    for roi in roi_defs:
        pid = roi["plaza_id"]
        x1, y1, x2, y2 = roi["bbox"]
        lbl = status.get(pid, "Libre")

        color = (0, 255, 0) if lbl == "Libre" else (0, 0, 255)
        if lbl == "Libre":
            libres += 1

        cv2.rectangle(vis, (x1, y1), (x2, y2), color, 2)
        cv2.putText(
            vis,
            f"{pid}:{'L' if lbl == 'Libre' else 'O'}",
            (x1 + 3, y1 + 15),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            color,
            1,
            cv2.LINE_AA,
        )

    header = f"Libres: {libres}/{total}"
    (tw, th), _ = cv2.getTextSize(header, cv2.FONT_HERSHEY_SIMPLEX, 1.2, 2)
    cv2.rectangle(vis, (10, 10), (10 + tw + 20, 10 + th + 20), (0, 0, 0), -1)
    cv2.putText(
        vis,
        header,
        (20, 10 + th + 10),
        cv2.FONT_HERSHEY_SIMPLEX,
        1.2,
        (255, 255, 255),
        2,
        cv2.LINE_AA,
    )
    return vis


Pipeline

In [18]:
def process_image(image_name: str,
                  iou_thr: float = 0.10,
                  conf_thr: float = 0.45):

    img_path = IM_DIR / image_name
    print("[DEBUG] Leyendo imagen:", img_path)
    if not img_path.exists():
        raise FileNotFoundError(f"No se encontró la imagen {img_path}")

    frame = cv2.imread(str(img_path))
    if frame is None:
        raise RuntimeError(f"No se pudo leer la imagen {img_path}")

    roi_defs = load_rois(image_name)
    print("[DEBUG] ROIs cargados:", len(roi_defs))

    boxes = detect_vehicles(frame, conf=conf_thr)
    status = occupancy_status(boxes, roi_defs, frame.shape, thr=iou_thr)
    print("[DEBUG] Plazas procesadas:", len(status))

    vis = draw_overlay(frame, roi_defs, status, boxes)

    stem = Path(image_name).stem
    overlay_path = RES_DIR / f"{stem}_overlay.jpg"
    cv2.imwrite(str(overlay_path), vis)

    rows = [{"image": image_name, "plaza_id": pid, "label": lbl}
            for pid, lbl in status.items()]
    df = pd.DataFrame(rows)
    csv_path = RES_DIR / f"{stem}_pred.csv"
    df.to_csv(csv_path, index=False)

    vis_rgb = cv2.cvtColor(vis, cv2.COLOR_BGR2RGB)

    libres = int((df.label == "Libre").sum())
    total = len(df)
    print(f"[OK] {image_name}: {libres} libres / {total} plazas")
    print("Overlay guardado en:", overlay_path)
    print("CSV guardado en    :", csv_path)

    return df, vis_rgb


In [19]:
def list_images_with_rois():
    imgs = []
    for ext in ("*.jpg", "*.jpeg", "*.png"):
        for p in IM_DIR.glob(ext):
            stem = p.stem
            roi_path = ROI_DIR / f"{stem}_rois.json"
            if roi_path.exists():
                imgs.append(p.name)
    imgs = sorted(imgs)
    print("[INFO] Imágenes registradas:", imgs)
    return imgs


def gr_detect_existing_image(image_name: str,
                             iou_thr: float = 0.10,
                             conf_thr: float = 0.45):
    if not image_name:
        return None, "Debes seleccionar una imagen."

    print("[DEBUG] Gradio: iniciando process_image para:", image_name)
    try:
        df, overlay_rgb = process_image(image_name, iou_thr=iou_thr, conf_thr=conf_thr)
        print("[DEBUG] Gradio: process_image terminó OK para:", image_name)
    except Exception as e:
        import traceback
        print("[ERROR] Gradio: process_image lanzó excepción:")
        traceback.print_exc()
        return None, f"ERROR al procesar {image_name}: {e}"

    libres = int((df["label"] == "Libre").sum())
    total = len(df)
    txt = f"Imagen: {image_name} | Plazas libres: {libres}/{total}"
    return overlay_rgb, txt


Interfaz Gradio

In [20]:
def build_gradio_app():
    with gr.Blocks(title="Sistema de detección de estacionamientos") as demo:
        gr.Markdown(
            "## Sistema de detección de estacionamientos (cámara fija)\n"
            "Selecciona una imagen registrada que tenga ROIs asociados."
        )

        with gr.Row():
            img_dropdown = gr.Dropdown(
                label="Imagen registrada",
                choices=list_images_with_rois(),
                value=None,
                interactive=True,
            )
            refresh_btn = gr.Button("🔄 Actualizar lista")

        detect_btn = gr.Button("Detectar plazas libres / ocupadas")

        with gr.Row():
            overlay_out = gr.Image(
                label="Resultado con overlay",
                type="numpy",
                interactive=False,
            )

        resumen_out = gr.Textbox(
            label="Resumen",
            interactive=False,
        )

        def _refresh_list():
            return gr.update(choices=list_images_with_rois())

        refresh_btn.click(_refresh_list, outputs=img_dropdown)
        detect_btn.click(
            fn=gr_detect_existing_image,
            inputs=img_dropdown,
            outputs=[overlay_out, resumen_out],
        )

    return demo


Uso

In [ ]:
demo = build_gradio_app()
demo.launch()

[INFO] Imágenes registradas: ['img_01.jpg', 'img_02.jpg', 'img_03.jpg', 'img_04.jpg', 'img_05.jpg', 'img_06.jpg', 'img_07.jpg', 'img_08.jpg', 'img_09.jpg', 'img_10.jpg']
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


[INFO] Imágenes registradas: ['img_01.jpg', 'img_02.jpg', 'img_03.jpg', 'img_04.jpg', 'img_05.jpg', 'img_06.jpg', 'img_07.jpg', 'img_08.jpg', 'img_09.jpg', 'img_10.jpg']
[DEBUG] Gradio: iniciando process_image para: img_02.jpg
[DEBUG] Leyendo imagen: E:\Adri\Universidad\II 2025\TOPICOS\Proyecto\parking-vision\data\images\img_02.jpg
[INFO] Cargados 29 ROIs desde img_02_rois.json
[DEBUG] ROIs cargados: 29
[DEBUG] Vehículos detectados: 21
[DEBUG] Plazas procesadas: 29
[OK] img_02.jpg: 16 libres / 29 plazas
Overlay guardado en: E:\Adri\Universidad\II 2025\TOPICOS\Proyecto\parking-vision\data\results\img_02_overlay.jpg
CSV guardado en    : E:\Adri\Universidad\II 2025\TOPICOS\Proyecto\parking-vision\data\results\img_02_pred.csv
[DEBUG] Gradio: process_image terminó OK para: img_02.jpg
